# Data Cleaning and Preparation of Fake Job Postings Data Set

<h3>Summary:</h3>

**Basic Description:**
- Original dataset contains 17880 entries and 18 columns

**Case cleaning:**
- n=281 duplicates have been removed
- n=1 with mainly missing values
- Final dataset contais n=17598 cases

**Text passages:**
- There are 4 different text passages of a job posting: company_profile, description, requirements, benefits.
- Dummy variables were created to capture whether a job states each of these passages.
- Based on these 4 passages, one overall text titled 'job_posting' was created.
- The character length of each text passage was stored

**Location:**
- The 'location' column was cleaned.
- For each row (where applicaple), country, state and city were extraxted.
- Dummies for countries with at least 1% occurence in the data were created.
- A new and shortend categorical column was created containing the most frequent countries.

**Salary Range:**
- Values were cleaned and splitted into columns 'from' and 'to'.
- Numerical columns capturing midpoint and range were created.
- Dummy column depicting whether a job posting states a salary range were created.

**Employment Type, Required Experience, and Education:**
- Dummies were created (One-Hot-Encoding).
- Education was grouped and a short categorical column was created.

In [1]:
# import libraries
import numpy as np
import os
import pandas as pd

In [2]:
# Adjust some pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

## Data Exploration

In [3]:
# import data and have a peek at the first 3 rows
df = pd.read_csv('C:/Users/nickm/Desktop/Coding/Projects/fake-job-postings/fake_job_postings.csv')
df.head(3)

job_id                                      title          location   
0       1                           Marketing Intern  US, NY, New York  \
1       2  Customer Service - Cloud Video Production    NZ, , Auckland   
2       3    Commissioning Machinery Assistant (CMA)     US, IA, Wever   

  department salary_range                                    company_profile   
0  Marketing          NaN  We're Food52, and we've created a groundbreaki...  \
1    Success          NaN  90 Seconds, the worlds Cloud Video Production ...   
2        NaN          NaN  Valor Services provides Workforce Solutions th...   

                                         description   
0  Food52, a fast-growing, James Beard Award-winn...  \
1  Organised - Focused - Vibrant - Awesome!Do you...   
2  Our client, located in Houston, is actively se...   

                                        requirements   
0  Experience with content management systems a m...  \
1  What we expect from you:Your key responsibilit...   
2  Implement pre-commissioning and commissioning ...   

                                            benefits  telecommuting   
0                                                NaN              0  \
1  What you will get from usThrough being part of...              0   
2                                                NaN              0   

   has_company_logo  has_questions employment_type required_experience   
0                 1              0           Other          Internship  \
1                 1              0       Full-time      Not Applicable   
2                 1              0             NaN                 NaN   

  required_education                   industry          function  fraudulent  
0                NaN                        NaN         Marketing           0  
1                NaN  Marketing and Advertising  Customer Service           0  
2                NaN                        NaN               NaN           0

In [4]:
# Display DataFrame summary
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17880 entries, 0 to 17879
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   job_id               17880 non-null  int64 
 1   title                17880 non-null  object
 2   location             17534 non-null  object
 3   department           6333 non-null   object
 4   salary_range         2868 non-null   object
 5   company_profile      14572 non-null  object
 6   description          17879 non-null  object
 7   requirements         15184 non-null  object
 8   benefits             10668 non-null  object
 9   telecommuting        17880 non-null  int64 
 10  has_company_logo     17880 non-null  int64 
 11  has_questions        17880 non-null  int64 
 12  employment_type      14409 non-null  object
 13  required_experience  10830 non-null  object
 14  required_education   9775 non-null   object
 15  industry             12977 non-null  object
 16  func

In [5]:
# Count number of unique values in each column
df.nunique()

job_id                 17880
title                  11231
location                3105
department              1337
salary_range             874
company_profile         1709
description            14801
requirements           11967
benefits                6204
telecommuting              2
has_company_logo           2
has_questions              2
employment_type            5
required_experience        7
required_education        13
industry                 131
function                  37
fraudulent                 2
dtype: int64

In [6]:
# Show number of missing values in columns
df.isna().sum()

job_id                     0
title                      0
location                 346
department             11547
salary_range           15012
company_profile         3308
description                1
requirements            2696
benefits                7212
telecommuting              0
has_company_logo           0
has_questions              0
employment_type         3471
required_experience     7050
required_education      8105
industry                4903
function                6455
fraudulent                 0
dtype: int64

In [7]:
# Store top 3 unique values in all columns
column = []
characteristic = []
valuecount = []

topx = 3

for c in df.columns:
    n = min(topx, len(df[c].value_counts()))
    column.extend(*[[c]*n])
    characteristic.extend(df[c].value_counts().head(topx).index)
    valuecount.extend(np.round(df[c].value_counts().head(topx).values/len(df)*100, 2))

In [8]:
# Show values
stats = pd.DataFrame({'column': column, 'characteristic': characteristic, 'valuecount_in_percent': valuecount})
stats

column                                     characteristic   
0                job_id                                                  1  \
1                job_id                                              11919   
2                job_id                                              11925   
3                 title                            English Teacher Abroad    
4                 title                        Customer Service Associate    
5                 title  Graduates: English Teacher Abroad (Conversatio...   
6              location                                    GB, LND, London   
7              location                                   US, NY, New York   
8              location                              US, CA, San Francisco   
9            department                                              Sales   
10           department                                        Engineering   
11           department                                          Marketing   
12         salary_range                                                0-0   
13         salary_range                                        40000-50000   
14         salary_range                                        30000-40000   
15      company_profile  We help teachers get safe &amp; secure jobs ab...   
16      company_profile  We Provide Full Time Permanent Positions for m...   
17      company_profile  Novitex Enterprise Solutions, formerly Pitney ...   
18          description  Play with kids, get paid for it Love travel? J...   
19          description  Play with kids, get paid for it :-)Love travel...   
20          description  Play with kids, get paid for it Love travel? J...   
21         requirements  University degree required. TEFL / TESOL / CEL...   
22         requirements  University degree required. TEFL / TESOL / CEL...   
23         requirements  16-18 year olds only due to government funding...   
24             benefits                                See job description   
25             benefits                                  Career prospects.   
26             benefits  CSD offers a competitive benefits package for ...   
27        telecommuting                                                  0   
28        telecommuting                                                  1   
29     has_company_logo                                                  1   
30     has_company_logo                                                  0   
31        has_questions                                                  0   
32        has_questions                                                  1   
33      employment_type                                          Full-time   
34      employment_type                                           Contract   
35      employment_type                                          Part-time   
36  required_experience                                   Mid-Senior level   
37  required_experience                                        Entry level   
38  required_experience                                          Associate   
39   required_education                                  Bachelor's Degree   
40   required_education                          High School or equivalent   
41   required_education                                        Unspecified   
42             industry                Information Technology and Services   
43             industry                                  Computer Software   
44             industry                                           Internet   
45             function                             Information Technology   
46             function                                              Sales   
47             function                                        Engineering   
48           fraudulent                                                  0   
49           fraudulent                                                  1   

    valuecount_in_percent  
0         

## Data Cleaning and Recoding

### Duplicates

In [9]:
# check for duplicated rows in df
df.drop('job_id', axis=1).duplicated().sum()

281

In [10]:
df.columns

Index(['job_id', 'title', 'location', 'department', 'salary_range',
       'company_profile', 'description', 'requirements', 'benefits',
       'telecommuting', 'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function',
       'fraudulent'],
      dtype='object')

In [11]:
# check for duplicated rows in df, ignoring job_id column
df.duplicated(subset=df.columns.difference(['job_id'])).sum()

281

In [12]:
# remove duplicates
print('#rows before duplicate removal:', len(df))
df = df.drop_duplicates(subset=df.columns.difference(['job_id']), ignore_index=True)
print('#rows after duplicate removal: ', len(df))

#rows before duplicate removal: 17880
#rows after duplicate removal:  17599


### Text presence

In [13]:
# define columns which contain descriptive texts of the job posting
cols = ['company_profile', 'description', 'requirements', 'benefits']

for c in cols:
    df[f'has_{c}'] = np.where(df[c].notna(), 1, 0)

In [14]:
# see columns
check_cols = cols + [f'has_{c}' for c in cols]
df[check_cols].head(3)

company_profile   
0  We're Food52, and we've created a groundbreaki...  \
1  90 Seconds, the worlds Cloud Video Production ...   
2  Valor Services provides Workforce Solutions th...   

                                         description   
0  Food52, a fast-growing, James Beard Award-winn...  \
1  Organised - Focused - Vibrant - Awesome!Do you...   
2  Our client, located in Houston, is actively se...   

                                        requirements   
0  Experience with content management systems a m...  \
1  What we expect from you:Your key responsibilit...   
2  Implement pre-commissioning and commissioning ...   

                                            benefits  has_company_profile   
0                                                NaN                    1  \
1  What you will get from usThrough being part of...                    1   
2                                                NaN                    1   

   has_description  has_requirements  has_benefits  
0                1                 1             0  
1                1                 1             1  
2                1                 1             0

In [15]:
# Check one case with missing 'description'
df.loc[df['description'].isna()]

job_id           title        location department salary_range   
17239   17514  Office Manager  PL, MZ, Warsaw        NaN          NaN  \

      company_profile description requirements benefits  telecommuting   
17239             NaN         NaN          NaN      NaN              0  \

       has_company_logo  has_questions employment_type required_experience   
17239                 0              0             NaN                 NaN  \

      required_education industry function  fraudulent  has_company_profile   
17239                NaN      NaN      NaN           1                    0  \

       has_description  has_requirements  has_benefits  
17239                0                 0             0

In [16]:
# remove this case as it mostly contains missings
print('#rows before case removal:', len(df))
df = df.drop(df[df['description'].isna()].index)
print('#rows after case removal: ', len(df))

#rows before case removal: 17599
#rows after case removal:  17598


### Text lengths 

In [17]:
# merge all columns with job texts into one column
df['job_posting'] = df[cols].fillna('').agg(' | '.join, axis=1)

In [18]:
# store character length of each text
cols.append('job_posting')

for c in cols:
    df[f'{c}_length'] = df[c].str.len()

In [19]:
# see columns
df.filter(like='_length').head()

company_profile_length  description_length  requirements_length   
0                  885.00                 905               852.00  \
1                 1286.00                2077              1433.00   
2                  879.00                 355              1363.00   
3                  614.00                2600              1429.00   
4                 1628.00                1520               757.00   

   benefits_length  job_posting_length  
0              NaN                2651  
1          1292.00                6097  
2              NaN                2606  
3           782.00                5434  
4            21.00                3935

In [20]:
# show descriptive statistics
df.filter(like='_length').describe()

company_profile_length  description_length  requirements_length   
count                14317.00            17598.00             14950.00  \
mean                   761.60             1217.27               696.15   
std                    536.96              895.29               609.21   
min                      9.00                6.00                 1.00   
25%                    391.00              604.00               303.00   
50%                    684.00             1017.00               562.00   
75%                   1022.00             1585.75               905.00   
max                   6178.00            14907.00             10864.00   

       benefits_length  job_posting_length  
count         10491.00            17598.00  
mean            350.17             2646.03  
std             376.46             1464.27  
min               1.00               18.00  
25%              92.00             1561.00  
50%             236.00             2499.00  
75%             485.00             3446.75  
max            4429.00            14916.00

### Country, State and City

In [21]:
# Break down the location column into country, state and city.
# E.g.: 'GB, LND, London'; 'US, NY, New York'
# -> Split column by comma
country_state_city = df['location'].str.split(pat=', ', expand=True)
country_state_city.head()

0   1           2     3     4     5     6     7     8     9     10    11   
0  US  NY    New York  None  None  None  None  None  None  None  None  None  \
1  NZ        Auckland  None  None  None  None  None  None  None  None  None   
2  US  IA       Wever  None  None  None  None  None  None  None  None  None   
3  US  DC  Washington  None  None  None  None  None  None  None  None  None   
4  US  FL  Fort Worth  None  None  None  None  None  None  None  None  None   

     12    13    14    15  
0  None  None  None  None  
1  None  None  None  None  
2  None  None  None  None  
3  None  None  None  None  
4  None  None  None  None

In [22]:
# For some postings, the format either differs, or there are multiple locations stated
country_state_city.sort_values(by=4).head()

0   1                  2    
4458   US  CA           San Jose  \
17295  US  FL  Orlando,Lake City   
4627   US  FL         Central Fl   
1718   US  TX             Austin   
4374   US  MI           Hartford   

                                                    3          4    
4458                                               SFO     Austin  \
17295  Jacksonville,Atlanta,Ocala,Miami,Asbury Park NJ  Belmar NJ   
4627                                           Nort Fl    Georgia   
1718                                       San Antonio    Houston   
4374                                           Paw Paw     Lawton   

                   5                                        6     7     8    
4458               OH                                     None  None  None  \
17295  Toms River NJ.                                     None  None  None   
4627          Alabama  Tennessee,South Carolina,North Carolina  None  None   
1718             None                                     None  None  None   
4374             None                                     None  None  None   

         9     10    11    12    13    14    15  
4458   None  None  None  None  None  None  None  
17295  None  None  None  None  None  None  None  
4627   None  None  None  None  None  None  None  
1718   None  None  None  None  None  None  None  
4374   None  None  None  None  None  None  None

In [23]:
# How many postings have more than one location (or differ in format)
len(df) - country_state_city.isna().sum()

0     17257
1     17166
2     17166
3       117
4        13
5         7
6         2
7         1
8         1
9         1
10        1
11        1
12        1
13        1
14        1
15        1
dtype: int64

In [24]:
# of those 118 postings that have more than 3 postings (see column 3), what information is stored in these extra columns?
country_state_city[[0,1,2,3]].loc[country_state_city[3].notna()].head(10)

0    1            2              3
16   IL          Tel Aviv         Israel
69   US   CA   Menlo Park             CA
98   US           Stocton             CA
214  US   CA  Bakersfield  CA / Mt. Poso
218  SA   01       Riyadh          Olaya
313  EG    C        Cairo      Nasr City
342  GR    I  Neo Iraklio         Athens
438  GB  UKM    Stockholm         Sweden
692  GB             Angel         London
977  US   CO      Boulder             CO

In [25]:
# The information in those extra columns seems to be mostly repetitve (e.g. the state again)
# Therefore, these columns will be ignored and only the first 3 columns will be used to determine
# the country, state and city of a job posting

df['location_country'] = np.where(country_state_city[0] != '', country_state_city[0], np.nan)
df['location_state'] = np.where(country_state_city[1] != '', country_state_city[1], np.nan)
df['location_city'] = np.where(country_state_city[2] != '', country_state_city[2], np.nan)

In [26]:
# show created location columns
df.filter(like='location').head()

location location_country location_state location_city
0    US, NY, New York               US             NY      New York
1      NZ, , Auckland               NZ            NaN      Auckland
2       US, IA, Wever               US             IA         Wever
3  US, DC, Washington               US             DC    Washington
4  US, FL, Fort Worth               US             FL    Fort Worth

In [27]:
# show value counts of unique countries present
country_top5 = [f'{value} ({count})' 
                for value, count 
                in zip(df['location_country'].value_counts().head().index,
                       df['location_country'].value_counts().head().values)]
state_top5 = [f'{value} ({count})'
              for value, count
              in zip(df['location_state'].value_counts().head().index,
                     df['location_state'].value_counts().head().values)]
city_top5 = [f'{value} ({count})'
             for value, count
             in zip(df['location_city'].value_counts().head().index,
                    df['location_city'].value_counts().head().values)]

pd.DataFrame({'Country Top 5' : country_top5,
              'State Top 5' : state_top5,
              'City Top 5' : city_top5})

Country Top 5 State Top 5           City Top 5
0    US (10460)   CA (2012)        London (1052)
1     GB (2329)   NY (1228)       New York (655)
2      GR (938)   LND (991)         Athens (541)
3      CA (450)    TX (957)  San Francisco (472)
4      DE (382)     I (686)        Houston (267)

In [28]:
# show value counts of unique states
df['location_state'].value_counts().head()

location_state
CA     2012
NY     1228
LND     991
TX      957
I       686
Name: count, dtype: int64

In [29]:
# show value counts of unique cities
df['location_city'].value_counts().head()

location_city
London           1052
New York          655
Athens            541
San Francisco     472
Houston           267
Name: count, dtype: int64

#### Create Country Dummy

In [30]:
# show countries with more than 1% occurance in dataset
country_frequency = df['location_country'].value_counts()
country_frequency_1pc = country_frequency[country_frequency.values >= len(df)*0.01]
print(country_frequency_1pc)
print('\nNumber of Categories:', len(country_frequency_1pc))

location_country
US    10460
GB     2329
GR      938
CA      450
DE      382
NZ      330
IN      273
AU      213
Name: count, dtype: int64

Number of Categories: 8


In [31]:
# create dummies for these 8 countries
dummy_countries = country_frequency_1pc.index.to_list()

for country in dummy_countries:
    df[f'location_country_{country}'] = np.where(df['location_country'] == country, 1, 0)
    df.loc[df['location_country'].isna(), f'location_country_{country}'] = np.nan

In [32]:
# create 'other' column
df['location_country_OTHER'] = np.where(~df['location_country'].isin(dummy_countries), 1, 0)
df.loc[df['location_country'].isna(), 'location_country_OTHER'] = np.nan

In [33]:
# check values
dummy_countries_columns = [f'location_country_{country}' for country in dummy_countries] + ['location_country_OTHER']

pd.DataFrame({'sum' : df[dummy_countries_columns].sum(),
              'NaNs' : df[dummy_countries_columns].isna().sum()})

sum  NaNs
location_country_US    10460.00   341
location_country_GB     2329.00   341
location_country_GR      938.00   341
location_country_CA      450.00   341
location_country_DE      382.00   341
location_country_NZ      330.00   341
location_country_IN      273.00   341
location_country_AU      213.00   341
location_country_OTHER  1882.00   341

#### Create Recoded Categorical Countries Column

In [34]:
df['location_country_short'] = np.where(((df['location_country'].isin(dummy_countries))
                                        | (df['location_country'].isna())), 
                                        df['location_country'], 
                                        'OTHER')

In [35]:
df['location_country_short'].value_counts()

location_country_short
US       10460
GB        2329
OTHER     1882
GR         938
CA         450
DE         382
NZ         330
IN         273
AU         213
Name: count, dtype: int64

### Salary Range

#### Quality Check and Cleaning

In [36]:
# Only 2868 of the 17880 job postings (~16%) have provided a salary range
print('salary_range not missing\nn:', df['salary_range'].notna().sum())
print('%:', round(df['salary_range'].notna().sum()/len(df)*100, 2), '%')

salary_range not missing
n: 2827
%: 16.06 %


In [37]:
# Show frequency of stated salary ranges
df.loc[df['salary_range'].notna(), 'salary_range'].value_counts().head(10)

salary_range
0-0             141
40000-50000      66
30000-40000      54
45000-67000      36
25000-30000      36
30000-50000      32
80000-100000     30
35000-45000      30
70000-90000      29
50000-70000      28
Name: count, dtype: int64

In [38]:
# There are some containing the value '0-0', which should be recorded to missing
df['salary_range'] = np.where(df['salary_range'] == '0-0', np.nan, df['salary_range'])

In [39]:
# Show sorted values
df.loc[df['salary_range'].notna(), 'salary_range'].sort_values()

3425          0-1
1593       0-1000
15884      0-1000
342        0-1000
10878    0-100000
           ...   
17377      Dec-25
10214      Jun-18
2304       Oct-15
14001      Oct-20
9819       Oct-20
Name: salary_range, Length: 2686, dtype: object

In [40]:
# There are values which are possibly incorrectly recoded and therefore show a date.
# E.g.: 'Dec-25', 'Oct-20', could be '12-25' or '10-20'
# Since we can't be sure, we'll exclude all values that contain any letter
df['salary_range'] = np.where(df['salary_range'].str.contains(r'[a-zA-Z]'), np.nan, df['salary_range'])

In [41]:
# Show sorted values again
df.loc[df['salary_range'].notna(), 'salary_range'].sort_values()

3425                0-1
342              0-1000
1593             0-1000
15884            0-1000
5292           0-100000
              ...      
2375     960000-1200000
16993    960000-1200000
13492           962-962
9571        99000-99999
8897        99000-99999
Name: salary_range, Length: 2661, dtype: object

In [42]:
# show first values of column
df.loc[df['salary_range'].notna(), 'salary_range'].head(10)

6       20000-28000
10    100000-120000
15    120000-150000
23    100000-120000
31      50000-65000
42      40000-50000
65            60-80
76      65000-70000
77           75-115
79     75000-110000
Name: salary_range, dtype: object

In [43]:
# Most of these values seem to depict the yearly range of salaries. However, some are
# either hourly ranges, or yearly ranges but in k [currency]
# E.g.: '60-80' or '75-115' 

#### Create new columns: 'salary_range_from' and 'salary_range_to'

##### Create Columns

In [44]:
# Let's split this column into 'from' and 'to'
df[['salary_range_from', 'salary_range_to']] = df['salary_range'].str.split(pat='-', expand=True).apply(pd.to_numeric)

In [45]:
# Show original and new columns
df.loc[df['salary_range'].notna()].filter(like='salary').head(10)

salary_range  salary_range_from  salary_range_to
6     20000-28000           20000.00         28000.00
10  100000-120000          100000.00        120000.00
15  120000-150000          120000.00        150000.00
23  100000-120000          100000.00        120000.00
31    50000-65000           50000.00         65000.00
42    40000-50000           40000.00         50000.00
65          60-80              60.00            80.00
76    65000-70000           65000.00         70000.00
77         75-115              75.00           115.00
79   75000-110000           75000.00        110000.00

##### Quality Check and Cleaning

In [46]:
# Show distribution
df.filter(like='salary_range_').describe()

salary_range_from  salary_range_to
count            2661.00          2660.00
mean           549873.08        866920.08
std          18284675.17      27962273.73
min                 0.00             1.00
25%             20000.00         30000.00
50%             40000.00         50000.00
75%             60000.00         90000.00
max         800000000.00    1200000000.00

In [47]:
# The max value seems extremely high. Let's have a closer look at this case in our DataFrame.
df.loc[df['salary_range']=='800000000-1200000000']

job_id                   title         location            department   
3126    3140  National Sales Manager  ID, JW, Jakarta  Danone - Sari Husada  \

              salary_range                                    company_profile   
3126  800000000-1200000000  Local expertise with an international perspect...  \

                                            description   
3126  Sari Husada is a member of a leading food and ...  \

                                           requirements benefits   
3126  The candidates will required to have a univers...  .../...  \

      telecommuting  has_company_logo  has_questions employment_type   
3126              0                 1              0       Full-time  \

     required_experience required_education        industry function   
3126    Mid-Senior level  Bachelor's Degree  Consumer Goods    Sales  \

      fraudulent  has_company_profile  has_description  has_requirements   
3126           0                    1                1                 1  \

      has_benefits                                        job_posting   
3126             1  Local expertise with an international perspect...  \

      company_profile_length  description_length  requirements_length   
3126                 1067.00                 733               608.00  \

      benefits_length  job_posting_length location_country location_state   
3126             7.00                2424               ID             JW  \

     location_city  location_country_US  location_country_GB   
3126       Jakarta                 0.00                 0.00  \

      location_country_GR  location_country_CA  location_country_DE   
3126                 0.00                 0.00                 0.00  \

      location_country_NZ  location_country_IN  location_country_AU   
3126                 0.00                 0.00                 0.00  \

      location_country_OTHER location_country_short  salary_range_from   
3126                    1.00                  OTHER       800000000.00  \

      salary_range_to  
3126    1200000000.00

In [48]:
# This case might state the salary range in Indonesian Rupiah. 
# Show top 3 values.
df.sort_values(by='salary_range_from', ascending=False).head(3)

job_id                                   title          location   
3126     3140                  National Sales Manager   ID, JW, Jakarta  \
12107   12252           Sales Manager - General Trade   ID, JW, Jakarta   
5049     5074  Junior Engineer/Engineering Technician  US, CA, San Jose   

                         department          salary_range   
3126           Danone - Sari Husada  800000000-1200000000  \
12107                      Arc Int.   500000000-800000000   
5049   Engineering, Pipeline Design       6266624-8582080   

                                         company_profile   
3126   Local expertise with an international perspect...  \
12107  Local expertise with an international perspect...   
5049                                                 NaN   

                                             description   
3126   Sari Husada is a member of a leading food and ...  \
12107  Client - ARC InternationalArc International is...   
5049    Founded in 1866, San Jose Water Company is an...   

                                            requirements   
3126   The candidates will required to have a univers...  \
12107  Driven and result orientedStrong in leadership...   
5049   Required Knowledge and Skills:Useful and profi...   

                                                benefits  telecommuting   
3126                                             .../...              0  \
12107                                                TBA              0   
5049   San Jose Water Company offers a very comprehen...              0   

       has_company_logo  has_questions employment_type required_experience   
3126                  1              0       Full-time    Mid-Senior level  \
12107                 1              0       Full-time    Mid-Senior level   
5049                  0              1       Full-time         Entry level   

      required_education        industry     function  fraudulent   
3126   Bachelor's Degree  Consumer Goods        Sales           0  \
12107  Bachelor's Degree  Consumer Goods        Sales           0   
5049    Associate Degree       Utilities  Engineering           1   

       has_company_profile  has_description  has_requirements  has_benefits   
3126                     1                1                 1             1  \
12107                    1                1                 1             1   
5049                     0                1                 1             1   

                                             job_posting   
3126   Local expertise with an international perspect...  \
12107  Local expertise with an international perspect...   
5049    |  Founded in 1866, San Jose Water Company is...   

       company_profile_length  description_length  requirements_length   
3126                  1067.00                 733               608.00  \
12107                 1067.00                1342               230.00   
5049                      NaN                1307              1058.00   

       benefits_length  job_posting_length location_country location_state   
3126              7.00                2424               ID             JW  \
12107             3.00                2651               ID             JW   
5049            168.00                2542               US             CA   

      location_city  location_country_US  location_country_GB   
3126        Jakarta                 0.00                 0.00  \
12107       Jakarta                 0.00                 0.00   
5049       San Jose                 1.00                 0.00   

       location_country_GR  location_country_CA  location_country_DE   
3126                  0.00                 0.00                 0.00  \
12107                 0.00                 0.00                 0.00   
5049                  0.00                 0.00                 0.00   

       location_country_NZ  location_country_IN  location_country_AU   
3126                  0.00                 0.00          

In [49]:
# The 2 highest (by far) are both from Indonesia.
# Inspect all salary_ranges from Indonesia
df.loc[(df['location_country']=='ID') & (df['salary_range'].notna())]

job_id                              title         location   
3126     3140             National Sales Manager  ID, JW, Jakarta  \
5878     5908  Become a co-Director in Indonesia      ID, , Medan   
9264     9342        Executive Search Consultant  ID, JW, Jakarta   
12107   12252      Sales Manager - General Trade  ID, JW, Jakarta   

                 department          salary_range   
3126   Danone - Sari Husada  800000000-1200000000  \
5878                    NaN               750-750   
9264   Wisma Metropolitan I           25000-35000   
12107              Arc Int.   500000000-800000000   

                                         company_profile   
3126   Local expertise with an international perspect...  \
5878                                                 NaN   
9264   Local expertise with an international perspect...   
12107  Local expertise with an international perspect...   

                                             description   
3126   Sari Husada is a member of a leading food and ...  \
5878   If you’re looking for a long-term volunteering...   
9264   Passionate about your Clients’ business, you a...   
12107  Client - ARC InternationalArc International is...   

                                            requirements   
3126   The candidates will required to have a univers...  \
5878   Those who are secure without the comforts of h...   
9264   Driven and Motivated, you are an out-going and...   
12107  Driven and result orientedStrong in leadership...   

                                                benefits  telecommuting   
3126                                             .../...              0  \
5878   Stipend: Volunteers must commit to serve at le...              0   
9264   Facing demanding Clients and extraordinary cha...              0   
12107                                                TBA              0   

       has_company_logo  has_questions employment_type required_experience   
3126                  1              0       Full-time    Mid-Senior level  \
5878                  0              0       Full-time      Not Applicable   
9264                  1              1       Full-time           Associate   
12107                 1              0       Full-time    Mid-Senior level   

      required_education                 industry    function  fraudulent   
3126   Bachelor's Degree           Consumer Goods       Sales           0  \
5878         Unspecified     Education Management   Education           0   
9264   Bachelor's Degree  Staffing and Recruiting  Consulting           0   
12107  Bachelor's Degree           Consumer Goods       Sales           0   

       has_company_profile  has_description  has_requirements  has_benefits   
3126                     1                1                 1             1  \
5878                     0                1                 1             1   
9264                     1                1                 1             1   
12107                    1                1                 1             1   

                                             job_posting   
3126   Local expertise with an international perspect...  \
5878    | If you’re looking for a long-term volunteer...   
9264   Local expertise with an international perspect...   
12107  Local expertise with an international perspect...   

       company_profile_length  description_length  requirements_length   
3126                  1067.00                 733               608.00  \
5878                      NaN                2299               607.00   
9264                  1067.00                 720               599.00   
12107                 1067.00                1342               230.00   

       benefits_length  job_posting_length location_country location_state   
3126              7.00                2424               ID             JW  \
5878            657.00                3572               ID            NaN   
9264            569.00               

In [50]:
# Let's convert the constructed values into USD for those cases.
# As of today (03/13/2024), the exchange rate is 1 USD = 15,574.35 IDR
idr_to_usd = 15_574.35
df['salary_range_from'] = np.where((df['location_country']=='ID') & (df['salary_range'].notna()),
                                   df['salary_range_from']/idr_to_usd,
                                   df['salary_range_from'])
df['salary_range_to'] = np.where((df['location_country']=='ID') & (df['salary_range'].notna()),
                                 df['salary_range_to']/idr_to_usd,
                                 df['salary_range_to'])

In [51]:
# Check results (values might be in hourly income, or yearly income in 1000 IDR)
df.loc[(df['location_country']=='ID') & (df['salary_range'].notna())].filter(like='salary')

salary_range  salary_range_from  salary_range_to
3126   800000000-1200000000           51366.51         77049.76
5878                750-750               0.05             0.05
9264            25000-35000               1.61             2.25
12107   500000000-800000000           32104.07         51366.51

In [52]:
# Check how many nonmissing values there are
df.filter(like='salary').notna().sum()

salary_range         2661
salary_range_from    2661
salary_range_to      2660
dtype: int64

In [53]:
# In one case we only have a value for the lower part of the range
df.loc[(df['salary_range_from'].notna()) & (df['salary_range_to'].isna())]

job_id                              title             location   
5511    5539  Inside Sales Magazine Advertising  US, CA, Yorba Linda  \

     department salary_range company_profile   
5511        NaN        40000             NaN  \

                                            description   
5511  ​IMPORTANT: TO BE CONSIDERED, PLEASE SEND YOUR...  \

                                           requirements   
5511  Desired Skills and ExperienceThe following att...  \

                                               benefits  telecommuting   
5511  This full time position includes a COMPETITIVE...              0  \

      has_company_logo  has_questions employment_type required_experience   
5511                 1              1       Full-time           Associate  \

     required_education    industry             function  fraudulent   
5511  Bachelor's Degree  Publishing  Accounting/Auditing           0  \

      has_company_profile  has_description  has_requirements  has_benefits   
5511                    0                1                 1             1  \

                                            job_posting   
5511   | ​IMPORTANT: TO BE CONSIDERED, PLEASE SEND Y...  \

      company_profile_length  description_length  requirements_length   
5511                     NaN                 340              2011.00  \

      benefits_length  job_posting_length location_country location_state   
5511           256.00                2616               US             CA  \

     location_city  location_country_US  location_country_GB   
5511   Yorba Linda                 1.00                 0.00  \

      location_country_GR  location_country_CA  location_country_DE   
5511                 0.00                 0.00                 0.00  \

      location_country_NZ  location_country_IN  location_country_AU   
5511                 0.00                 0.00                 0.00  \

      location_country_OTHER location_country_short  salary_range_from   
5511                    0.00                     US           40000.00  \

      salary_range_to  
5511              NaN

In [54]:
# Store ID for recode and check
check_id = df.loc[(df['salary_range_from'].notna()) & (df['salary_range_to'].isna())].index
check_id

Index([5511], dtype='int64')

In [55]:
# This job offer stated the anticipated final pay instead of a range.
# We'll just use this value to fill the 'to' column
df.loc[check_id, 'salary_range_to'] = df.loc[check_id, 'salary_range_from']
df.filter(like='salary').iloc[check_id]

salary_range  salary_range_from  salary_range_to
5511        40000           40000.00         40000.00

#### Create new columns: 'salary_range_mid' and 'salary_range_peak_to_peak'

In [56]:
# Create a column containing the midpoint of the stated salary range.
df['salary_range_mid'] = np.mean(df[['salary_range_from', 'salary_range_to']], axis=1)

In [57]:
# Create a column containing the range (max-min) of the salary range.
df['salary_range_peak_to_peak'] = np.ptp(df[['salary_range_from', 'salary_range_to']], axis=1)

In [58]:
# Show columns
df.filter(like='salary').loc[df['salary_range'].notna()].head()

salary_range  salary_range_from  salary_range_to  salary_range_mid   
6     20000-28000           20000.00         28000.00          24000.00  \
10  100000-120000          100000.00        120000.00         110000.00   
15  120000-150000          120000.00        150000.00         135000.00   
23  100000-120000          100000.00        120000.00         110000.00   
31    50000-65000           50000.00         65000.00          57500.00   

    salary_range_peak_to_peak  
6                     8000.00  
10                   20000.00  
15                   30000.00  
23                   20000.00  
31                   15000.00

#### Create new column: 'has_salary_range'

In [59]:
# Create binary variable depicting whether a salary range is stated in the job listing.
df['has_salary_range'] = np.where(df['salary_range'].notna(), 1, 0)
df['has_salary_range'].value_counts()

has_salary_range
0    14937
1     2661
Name: count, dtype: int64

### One Hot Encoding of 'employment_type', 'required_experience' and 'required_education'

In [60]:
# Define columns to work with
one_hot_columns = ['employment_type', 'required_experience', 'required_education']

In [61]:
# Show unique values and corresponding counts of these columns
for col in one_hot_columns:
    print(df[col].value_counts(dropna=False), '\n')

employment_type
Full-time    11423
NaN           3427
Contract      1514
Part-time      772
Temporary      237
Other          225
Name: count, dtype: int64 

required_experience
NaN                 6961
Mid-Senior level    3768
Entry level         2637
Associate           2267
Not Applicable      1070
Director             383
Internship           372
Executive            140
Name: count, dtype: int64 

required_education
NaN                                  8008
Bachelor's Degree                    5099
High School or equivalent            1989
Unspecified                          1369
Master's Degree                       416
Associate Degree                      264
Certification                         165
Some College Coursework Completed     100
Professional                           73
Vocational                             47
Some High School Coursework            27
Doctorate                              26
Vocational - HS Diploma                 9
Vocational - Degree          

In [62]:
# One hot encoding, not considering NaNs
dummies = pd.get_dummies(df[one_hot_columns], dummy_na=False, dtype=int)

In [63]:
# recode NaNs in dummy columns
for df_column in one_hot_columns:
    dummy_columns = dummies.filter(like=f'{df_column}_').columns
    for dummy_column in dummy_columns:
        dummies.loc[df[df_column].isna(), dummy_column] = np.nan

In [64]:
# Merge dummies to df
df = pd.concat([df, dummies], axis=1)

In [65]:
# Check columns
df[dummies.columns].sum()

employment_type_Contract                                1514.00
employment_type_Full-time                              11423.00
employment_type_Other                                    225.00
employment_type_Part-time                                772.00
employment_type_Temporary                                237.00
required_experience_Associate                           2267.00
required_experience_Director                             383.00
required_experience_Entry level                         2637.00
required_experience_Executive                            140.00
required_experience_Internship                           372.00
required_experience_Mid-Senior level                    3768.00
required_experience_Not Applicable                      1070.00
required_education_Associate Degree                      264.00
required_education_Bachelor's Degree                    5099.00
required_education_Certification                         165.00
required_education_Doctorate            

#### Combine Education Columns

In [66]:
# combine professional education into one feature
professional_education_columns = ['required_education_Certification',
                                  'required_education_Professional',
                                  'required_education_Vocational', 
                                  'required_education_Vocational - Degree',
                                  'required_education_Vocational - HS Diploma']

df['required_education_Prof_Voca_Cert'] = np.any(df[professional_education_columns], axis=1).astype(float)

In [67]:
# recode NaNs
df.loc[df['required_education'].isna(), 'required_education_Prof_Voca_Cert'] = np.nan

In [68]:
# Check count in merged column
print('Total in all individual columns:', df[professional_education_columns].sum().sum())
print('Total in created single column: ', df['required_education_Prof_Voca_Cert'].sum())

Total in all individual columns: 300.0
Total in created single column:  300.0


In [69]:
# remove original columns
df = df.drop(professional_education_columns, axis=1)

#### Create New/Short Categorical Education Column

In [70]:
# define labels to be recoded to 'Professional, Vocational or Certified'
professional_education_labels = ['Certification',
                                 'Professional',
                                 'Vocational', 
                                 'Vocational - Degree',
                                 'Vocational - HS Diploma']

df['required_education_short'] = np.where(((~df['required_education'].isin(professional_education_labels))
                                           | (df['required_education'].isna())),
                                           df['required_education'], 
                                           'Professional, Vocational or Certified')

In [71]:
# show resulting value count
df['required_education_short'].value_counts(dropna=False)

required_education_short
NaN                                      8008
Bachelor's Degree                        5099
High School or equivalent                1989
Unspecified                              1369
Master's Degree                           416
Professional, Vocational or Certified     300
Associate Degree                          264
Some College Coursework Completed         100
Some High School Coursework                27
Doctorate                                  26
Name: count, dtype: int64

## Store DF for further analyses

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17598 entries, 0 to 17598
Data columns (total 68 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   job_id                                                17598 non-null  int64  
 1   title                                                 17598 non-null  object 
 2   location                                              17257 non-null  object 
 3   department                                            6271 non-null   object 
 4   salary_range                                          2661 non-null   object 
 5   company_profile                                       14317 non-null  object 
 6   description                                           17598 non-null  object 
 7   requirements                                          14950 non-null  object 
 8   benefits                                              10491 n

In [73]:
df.to_csv('fake_job_postings_CLEANED_RECODED.csv', encoding='utf-8', index=False)
df.to_csv('fake_job_postings_CLEANED_RECODED.gzip', encoding='utf-8', index=False, compression='gzip')

In [74]:
# Compare file sizes
csv_filesize = os.path.getsize('fake_job_postings_CLEANED_RECODED.csv') / 1024**2
gzip_filesize = os.path.getsize('fake_job_postings_CLEANED_RECODED.gzip') / 1024**2

In [75]:
print(f'csv:  {csv_filesize:.2f} MB')
print(f'gzip: {gzip_filesize:.2f} MB')

csv:  94.75 MB
gzip: 18.36 MB
